In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import vectorbt as vbt
import ta

In [ ]:
BTC=pd.read_csv('BTCUSDT.csv')
BTC=BTC.set_index("Date")


In [ ]:
def Technical_indicators(Merged,target):
       
    window_4_hr=60*4
    window_30_M=30
    window_1_day=60*24
    window_2_day=60*24*2
    window_2_weeks=60*24*14

    Merged['ema 4HR'] = Merged[target].ewm(span=window_4_hr, adjust=False).mean()
    Merged['ema 1D'] = Merged[target].ewm(span=window_1_day, adjust=False).mean()
    Merged['ema 2W'] = Merged[target].ewm(span=window_2_weeks, adjust=False).mean()
    Merged['SMA_50'] = ta.trend.sma_indicator(Merged[target], window=50)
    Merged['SMA_200'] = ta.trend.sma_indicator(Merged[target], window=200)
    Merged['SMA_12HR'] = ta.trend.sma_indicator(Merged[target], window=60*12)
    return Merged

In [ ]:
Merged=Technical_indicators(BTC,'Close')

In [ ]:
Merged[['Close', 'SMA_50', 'SMA_200',]].iloc[-60*30*10:].plot()

In [ ]:
'''
ema1>Close sell
ema1<Close buy
SMA50>SMA200 Buy

'''

In [ ]:
Merged.keys()

In [ ]:
60000

In [ ]:
Merged['Signal']=np.where((Merged['SMA_50']>Merged['SMA_200']*1.01) & (Merged['ema 1D']*1.01<Merged['Close']),1,
                          np.where((Merged['SMA_50']*1.01<Merged['SMA_200']) & (Merged['ema 1D']>Merged['Close']*1.01),-1,0))

Merged['Signal'].unique()

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10, 10))

Merged['Close'][200000:500000].plot(ax=ax)
Merged['ema 1D'][200000:500000].plot(ax=ax, style='--')
Merged['SMA_50'][200000:500000].plot(ax=ax, style='--')
Merged['SMA_200'][200000:500000].plot(ax=ax, style='--')
ax.legend()
ax2 = ax.twinx()
Merged['Signal'][200000:500000].plot(ax=ax2, color='green', label='Signal')
ax2.legend()


In [ ]:
buy_signals = Merged['Signal']==1
# sell_signal=Merged['Signal']==-1
# Create the portfolio
pf = vbt.Portfolio.from_signals(Merged['Close'],
                                       entries=buy_signals,
                                    #    exits=sell_signal,
                                       # tp_stop=0.2,
                                       sl_stop=0.1,
                                       sl_trail=True,
                                    #    short_entries=sell_signal,
                                    #    short_exits=buy_signals,
                                       fees=0.005,init_cash=10000)
# Analyze the results
portfolio_perf = pf.stats()
# print(np.round(portfolio_perf,2))
print((portfolio_perf))

In [ ]:
pf.plot().show()